# Traces with network manager
This notebook just illustrates the role of the time constants in the traces and furthermore works as a function test of the new class. The network manager.

In [1]:
from __future__ import print_function
import pprint
import subprocess
import sys 
sys.path.append('../')

import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from mpl_toolkits.axes_grid1 import make_axes_locatable

from network import BCPNN, NetworkManager
from data_transformer import build_ortogonal_patterns
from analysis_functions import calculate_angle_from_history
from analysis_functions import calculate_winning_pattern_from_distances, calculate_patterns_timings

%matplotlib inline
matplotlib.rcParams.update({'font.size': 22})

np.set_printoptions(suppress=True, precision=2)

In [ ]:
run_old_version = False
if run_old_version:
    hash_when_file_was_written = 'bb3e2db5733040a57f083206ad03fb83b1605c2a'
    hash_at_the_moment = subprocess.check_output(["git", 'rev-parse', 'HEAD']).strip()
    print('Actual hash', hash_at_the_moment)
    print('Hash of the commit used to run the simulation', hash_when_file_was_written)
    subprocess.call(['git', 'checkout', hash_when_file_was_written])

#### Network
We build the patterns and the network

In [2]:
hypercolumns = 2
minicolumns = 5
n_patterns = 3  # Number of patterns

patterns_dic = build_ortogonal_patterns(hypercolumns, minicolumns)
patterns = list(patterns_dic.values())
patterns = patterns[:n_patterns]

# Build the network
tau_z_pre = 0.500
tau_z_post = 0.050

nn = BCPNN(hypercolumns, minicolumns, tau_z_post=tau_z_post, tau_z_pre=tau_z_pre)
nn.randomize_pattern()

Now we build the network manager and run the model in training mode with the patterns that we already build

In [3]:
dt = 0.001
T_training = 2.0
training_time = np.arange(0, T_training + dt, dt)
values_to_save = ['o', 'z_pre', 'z_post', 'a']
manager = NetworkManager(nn=nn, time=training_time, values_to_save=values_to_save)

for pattern in patterns:
    nn.k = 1.0
    print('trained')
    # history = nn.run_network_simulation(time=training_time, I=pattern, save=True)
    manager.run_network(time=training_time, I=pattern)
    manager.run_network(time=training_time)

trained
trained
trained


#### Plots
We extract the information and plot the different 